In [1]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
from astropy.table import Table
from astropy import table
from astroquery.gaia import Gaia
from scipy.optimize import minimize
from scipy.stats import scoreatpercentile
from scipy.interpolate import LinearNDInterpolator, NearestNDInterpolator, RBFInterpolator
import stam
from tqdm import tqdm
import WD_models
from os import path
import extinction
from astropy.constants import R_sun, G, M_sun
import astropy.units as u
%matplotlib tk

# Sources

In [5]:
# clusters = Table.read('../data/hunt_clusters/clusters.csv')
# members = Table.read('../data/hunt_clusters/members.csv')
sources = Table.read('../table_C.fits', format='fits')

In [6]:
# from astroquery.vizier import Vizier
# clstrs = Vizier(catalog='J/MNRAS/504/356',columns=['Cluster','N','logage','e_logage','[Fe/H]','e_[Fe/H]','Av','e_Av','FileName'],row_limit=-1).query_constraints()[0]

# clstrs.rename_columns(clstrs.colnames,['Cluster','N','logage','e_logage','Fe/H','e_Fe/H','Av','e_Av','FileName'])
# # sources = Table.read('../table_extra.fits')
# sources['idx'] = np.full(len(sources),'---',dtype=object)

# cut = (sources['nss_solution_type'] == 'Orbital') | (sources['nss_solution_type'] == 'AstroSpectroSB1')
# sources = sources[cut]

# nms = np.unique(sources['cluster'])
# for n in nms:
#     for i, s in enumerate(sources[sources['cluster'] == n]):
#         j = np.where(sources['source_id'] == s['source_id'])[0][0]
#         tbl[j]['idx'] = n+'_'+str(i)

# sources['idx'] = sources['idx'].astype(str)
# sources['age'] = np.full(len(sources),np.nan)
# sources['[Fe/H]'] = np.full(len(sources),np.nan)
# sources['Av'] = np.full(len(sources),np.nan)

# for j in range(len(sources)):
#     if sources[j]['cluster'] == 'Hyades':
#         sources[j]['age'] = 680
#         sources[j]['[Fe/H]'] = 0.14
#         sources[j]['Av'] = 0.01 * 3.1
#     if sources[j]['cluster'] == 'IC_2602':
#         sources[j]['age'] = 60
#         sources[j]['[Fe/H]'] = -0.02
#         sources[j]['Av'] = 0.05 * 3.1
#     if sources[j]['cluster'] == 'NGC_2287':
#         sources[j]['age'] = 200
#         sources[j]['[Fe/H]'] = -0.11
#         sources[j]['Av'] = 0.03 * 3.1
#     if sources[j]['cluster'] == 'NGC_2547':
#         sources[j]['mh'] = 60
#         sources[j]['[Fe/H]'] = 0.01
#         sources[j]['Av'] = 0.06 * 3.1

# for j in range(len(sources)):
#     clstr = sources[j]['cluster']
#     if clstr in clstrs['cluster']:
#         i = np.where(clstrs['cluster'] == clstr)[0][0]
#         sources[j]['age'] = clstrs[i]['age']
#         sources[j]['[Fe/H]'] = clstrs[i]['Fe_H']
#         sources[j]['Av'] = clstrs[i]['ebv'] * 3.1
    
# sources.sort('idx')

# PARSEC functions

In [73]:
# PARSEC_path = '../data/PARSEC v1.2S/Gaia_lin/'
PARSEC_path = '../OCFit/gaiaDR2/grids/'
models = stam.getmodels.read_parsec(path=PARSEC_path)


def choose_model(mh):
    if (-0.6 <= mh) and (mh <= 0.05):
        PARSEC_path = '../data/PARSECv2.0/w_i=0.6/'
        models = stam.getmodels.read_parsec(path=PARSEC_path)
    else:
        PARSEC_path = '../data/PARSEC v1.2S/Gaia_lin/'
        models = stam.getmodels.read_parsec(path=PARSEC_path)
    return PARSEC_path,models

def get_tracks(mh,age):
    
    color_fil_1, color_fil_2, mag_fil = "G_BP", "G_RP", "G" ## no rotation    
    if PARSEC_path == '../data/PARSECv2.0/w_i=0.6/':
        color_fil_1, color_fil_2, mag_fil = "G_BP_i45", "G_RP_i45", "G_i45" ## with rotation
    age_res = np.min(abs(10**np.unique(models['logAge'].data) * 1e-9 - age * 1e-3)) * 1.05 + 1e-3
    mh_res = np.min(abs(np.unique(models['MH'].data) - mh)) + 0.005
   
    stage_min = 0  # pre-main sequence
    stage_max = 5  # red giant branch
    mass_min = 0  # [Msun]
    mass_max = 30  # [Msun]
    tracks = stam.gentracks.get_isotrack(models, [age * 1e-3, mh], params=("age", "mh"), return_table=True,
                                    age_res=age_res, mass_min=mass_min, mass_max=mass_max, mh_res = mh_res,
                                    stage=None, stage_min=stage_min, stage_max=stage_max, sort_by="age", color_filter1=color_fil_1, color_filter2=color_fil_2,
                mag_filter=mag_fil)
    return tracks

def get_track_idx(mh,age):
    color_fil_1, color_fil_2, mag_fil = "G_BP", "G_RP", "G" ## no rotation    
    age_res = np.min(abs(10**np.unique(models['logAge'].data) * 1e-9 - age * 1e-3)) * 1.05
    stage_min = 0  # pre-main sequence
    stage_max = 5  # red giant branch
    mass_min = 0  # [Msun]
    mass_max = 8  # [Msun]
    track_idx = stam.gentracks.get_isotrack(models, [age * 1e-3, mh], params=("age", "mh"), return_idx=True,
                                    age_res=age_res, mass_min=mass_min, mass_max=mass_max, mh_res = 0.025,
                                    stage=None, stage_min=stage_min, stage_max=stage_max, sort_by="age", color_filter1=color_fil_1, color_filter2=color_fil_2,
                mag_filter=mag_fil)   
    return track_idx

def get_age_mh_grid():
    age = 10**np.unique(models['logAge'].data) * 1e-6
    mh = np.unique(models['MH'].data)
    age,mh = np.meshgrid(age,mh)
    return age,mh


# c = (models['label'] <= 5) & (models['Mini']<=8)
# p1 = models[c]['Mass']
# p2 = models[c]['logAge']
# p3 = models[c]['MH']
# pdep1 = models[c]['G_BPmag'] - models[c]['G_RPmag']
# pdep2 = models[c]['Gmag']

# mass_to_mag = LinearNDInterpolator(np.vstack(list(zip(p1,p2,p3))), pdep2)
# mass_to_color = LinearNDInterpolator(np.vstack(list(zip(p1,p2,p3))), pdep1)

# def get_isochrone(mh,age):
#     mass_arr = get_tracks(mh,age)['mass']
#     gmag = mass_to_mag(mass_arr,np.log10(age*1e6),mh)
#     bprp = mass_to_color(mass_arr,np.log10(age*1e6),mh)
#     return Table([mass_arr,bprp,gmag],names=['mass','bp_rp','mg'])

# _ = get_isochrone(0,101)

# Cluster parameters and primary mass

In [5]:
## mass interpolations, cluster cmd plots

def tracks2grid(tracks, xparam = "bp_rp", yparam = "mg", xstep=0.05, ystep=0.05):
    # auxiliary to interp_mass_realization
    xmin = np.min(np.around(tracks[xparam], -int(np.round(np.log10(xstep)))))
    xmax = np.max(np.around(tracks[xparam], -int(np.round(np.log10(xstep)))))
    ymin = np.min(np.around(tracks[yparam], -int(np.round(np.log10(ystep)))))
    ymax = np.max(np.around(tracks[yparam], -int(np.round(np.log10(ystep)))))
    x, y = np.meshgrid(np.arange(xmin, xmax, xstep), np.arange(ymin, ymax, ystep))
            
    return x, y, xmin, xmax, ymin, ymax

def interp_mass_realization(age,mh,av,bp_rp,mg):
    # For a single realization of cluster and source parameters, interpolate the mass

    # get the isotrack for this realization of the cluster parameters
    tracks = get_tracks(mh,age)
    x = np.array(tracks["bp_rp"])
    y = np.array(tracks["mg"])
    z = np.array(tracks["mass"])
    # Add small noise to the isochrone (to avoid singularity problems)
    x = x + np.random.normal(0, 0.0001, len(x))
    y = y + np.random.normal(0, 0.0001, len(y))
    z = z + np.random.normal(0, 0.0001, len(z))
    # apply extinction to the isochrone
    ag, e_bprp = extinction.get_AG_EBPRP(av,x)
    y = y + ag
    x = x + e_bprp
    # interpolate this instance of color and magnitude along the isochrone
    xstep, ystep = 0.05, 0.05
    grid_x, grid_y, xmin, xmax, ymin, ymax = tracks2grid(tracks, xstep=xstep, ystep=ystep)    
    fun_type = "linear"
    interp = RBFInterpolator(np.array([x, y]).T, z, kernel=fun_type)
    grid = np.array([grid_x, grid_y])
    grid_flat = grid.reshape(2, -1).T
    grid_z = interp(grid_flat).reshape(grid_x.shape)
    mass = interp(np.array([[bp_rp, mg]]))[0]
    return mass

def photometric_mass(idx,save=False,plot=False,n_realizations=100):        
    # Run monte carlo simulation to estimate primary pass and error
    j = np.where(sources['idx'] == idx)[0][0]
    age, e_age = sources[j]['age'], sources[j]['e_age']
    mh, e_mh = sources[j]['[Fe/H]'], sources[j]['e_[Fe/H]']
    av, e_av = sources[j]['Av'], sources[j]['e_Av']
    mg = sources[j]['mg']
    bp_rp = sources[j]['bp_rp']

    # estimate photometric errors
    e_g= 2.5*np.log(10)*sources[j]['phot_g_mean_flux_error']/sources[j]['phot_g_mean_flux']
    e_mg = np.sqrt(e_g**2 + (2.17 / sources[j]['parallax_over_error'])**2)
    e_bp = 2.5 * np.log(10) * sources[j]['phot_bp_mean_flux_error'] / sources[j]['phot_bp_mean_flux']
    e_rp = 2.5 * np.log(10) * sources[j]['phot_rp_mean_flux_error'] / sources[j]['phot_rp_mean_flux']
    e_bp_rp = np.sqrt(e_bp**2 + e_rp**2)

    # run monte carlo simulation
    age_vec = np.random.normal(age,e_age,n_realizations)
    mh_vec = np.random.normal(mh,e_mh,n_realizations)
    av_vec = np.random.normal(av,e_av,n_realizations)
    bp_rp_vec = np.random.normal(bp_rp,e_bp_rp,n_realizations)
    mg_vec = np.random.normal(mg,e_mg,n_realizations)
    mass_vec = np.zeros(n_realizations)
    for i in range(n_realizations):
        try:
            mass_vec[i] = interp_mass_realization(age_vec[i],mh_vec[i],av_vec[i],bp_rp_vec[i],mg_vec[i])
        except:
            mass_vec[i] = np.nan
    m1 = np.nanmean(mass_vec)
    m1_err = np.nanstd(mass_vec)
    plot_mass_interp(age,mh,av,bp_rp,mg,m1,m1_err,idx,sources[j]['cluster'],save,plot)
    return m1,m1_err

def get_cluster_members(cluster):
    # read the cluster member data from our premade files, apply quality cuts

    filepath = path.join('..','OCFit','gaiaDR2','data',cluster+'.dat')
    obs = np.genfromtxt(filepath,names=True,delimiter=',',dtype=None)
    if path.exists(filepath.replace('.dat','_D.dat')):
        obsD = np.genfromtxt(filepath.replace('.dat','_D.dat'),names=True,delimiter=',',dtype=None)
        obs = np.concatenate((obs,obsD))

    mg = obs['Gmag'] - 5 * np.log10(1000/obs['Plx']) + 5
    #remove nans para fazer os plots
    cond1 = np.isfinite(obs['Gmag'])
    cond2 = np.isfinite(obs['BPmag'])
    cond3 = np.isfinite(obs['RPmag'])
    
    cond4 = obs['RFG'] > 50.0
    cond5 = obs['RFBP'] > 20.0
    cond6 = obs['RFRP'] > 20.0
    cond7 = obs['E_BR_RP_'] < 1.3+0.06*(obs['BPRP'])**2
    cond8 = obs['E_BR_RP_'] > 1.0+0.015*(obs['BPRP'])**2
    cond9 = obs['Nper'] > 8
    cond10 = obs['fidelity_v2'] > 0.5
    cond11 = (mg < 9) | (obs['BPRP'] > 0)       
    ind  = np.where(cond1&cond2&cond3&cond4&cond5&cond6&cond7&cond8&cond9&cond10&cond11)
    obs = obs[ind]
    obs = Table(obs)
    obs = table.unique(obs,keys='source_id')
    return obs
    
def plot_cmd(idx,cluster,age,mh,ebv,save,plot):
        memb = get_cluster_members(cluster)
        memb['mg'] = memb['Gmag'] - 5 * np.log10(1000/memb['Plx']) + 5

        mg = memb['mg']
        bp_rp = memb['BPRP']
        mg0 = sources[sources['idx']== idx]['mg']
        bp_rp0= sources[sources['idx']== idx]['bp_rp']

        track = get_tracks(mh,age)
        bp_rp_model = np.array(track['bp_rp'])
        mg_model = np.array(track['mg'])
        ag, e_bprp = extinction.get_AG_EBPRP(3.1*ebv,bp_rp_model)
        mg_model = mg_model + ag
        bp_rp_model = bp_rp_model + e_bprp

        fig,ax = plt.subplots(dpi=300)
        ax.scatter(bp_rp,mg,s=5,c='r',label='Cluster members',zorder=5)
        ax.scatter(bp_rp0,mg0,s=25,c='g',label='Candidate',zorder = 10)
        ax.plot(bp_rp_model,mg_model, 'ko', markersize=1,label=f'PARSEC age={int(age)} Myr, [Fe/H]={mh:.2f}, E(B-V)={ebv:.3f}')
        ax.set_xlabel('BP-RP')
        ax.set_ylabel('G')
        ax.invert_yaxis()
        ax.legend(loc='lower left',frameon=False)
        ax.set_title('Candidate '+ str(idx) + ' in cluster ' + cluster)
        if save:
            fig.savefig(f'../img/cmd/{idx}_'+cluster+'.png')
        if not plot:
            plt.close()

def plot_mass_interp(age,mh,av,bp_rp,mg,m1,m1_err,idx,clstr,save,plot):
    # For the final mass estimate, plot the mass interpolation
    tracks = get_tracks(mh,age)
    x = np.array(tracks["bp_rp"])
    y = np.array(tracks["mg"])
    z = np.array(tracks["mass"])
    # Add small noise to the isochrone (to avoid singularity problems)
    x = x + np.random.normal(0, 0.0001, len(x))
    y = y + np.random.normal(0, 0.0001, len(y))
    z = z + np.random.normal(0, 0.0001, len(z))
    # apply extinction to the isochrone
    ag, e_bprp = extinction.get_AG_EBPRP(av,x)
    y = y + ag
    x = x + e_bprp
    # interpolate this instance of color and magnitude along the isochrone
    xstep, ystep = 0.05, 0.05
    grid_x, grid_y, xmin, xmax, ymin, ymax = tracks2grid(tracks, xstep=xstep, ystep=ystep)    
    fun_type = "linear"
    interp = RBFInterpolator(np.array([x, y]).T, z, kernel=fun_type)
    grid = np.array([grid_x, grid_y])
    grid_flat = grid.reshape(2, -1).T
    grid_z = interp(grid_flat).reshape(grid_x.shape)
    # plot
    fig, ax = plt.subplots(dpi=300, layout='constrained')
    ax.plot(x[z>0.3], y[z>0.3], 'ko', markersize=1,label=f'PARSEC age={int(age)} Myr, [Fe/H]={mh:.2f}, E(B-V)={av/3.1:.3f}')
    h = ax.imshow(grid_z, origin="lower", extent=[xmin, xmax, ymin, ymax], cmap='Oranges', aspect='auto')
    ax.scatter(bp_rp,mg,s=50,c='b',label=f'Candidate {idx} $M_1$={m1:.2f} $\pm$ {m1_err:.2f} $M_\odot$',zorder = 10)
    cbar = plt.colorbar(h)
    cbar.set_label('Mass [$M_\odot$]',fontsize = 12)
    ax.set_xlabel(r"$G_\text{BP}-G_\text{RP}$", fontsize=12)
    ax.set_ylabel(r"$G$", fontsize=12)
    ax.set_xlim(xmin, xmax)
    ax.set_ylim(ymin, ymax)
    # ax.set_title(f'Candidate {idx} in cluster {clstr}')
    ax.invert_yaxis()
    ax.legend(loc='lower left',frameon=False,fontsize=11)
    
    if save:
        fig.savefig(f'../img/mass_interp/{idx}_'+clstr+'.png')
    if not plot:
        plt.close()

In [138]:
plot = False
save = True
sources = Table.read('../table_C.fits')
new_sources = sources.copy()
m1_col = []
m1_err_col = []

# fitting all with parsec v1.2S
# PARSEC_path = '../data/PARSEC v1.2S/Gaia_lin/'
PARSEC_path = '../OCFit/gaiaDR2/grids/'
models = stam.getmodels.read_parsec(path=PARSEC_path)
# for idx in new_sources['idx']:
for idx in sources['idx']:
    age = new_sources[new_sources['idx'] == idx]['age'][0]
    mh = new_sources[new_sources['idx'] == idx]['[Fe/H]'][0]
    ebv = new_sources[new_sources['idx'] == idx]['Av'][0] / 3.1
    cluster = new_sources[new_sources['idx'] == idx]['cluster'][0]
    if np.isnan(age):
        # if unable to fit age, skip the candidate
        # new_sources.remove_row(np.where(new_sources['idx'] == idx)[0][0])
        m1_col.append(np.nan)
        m1_err_col.append(np.nan)
    else:
        try:
            # print(f'Photometric mass for candidate {idx}...')
            # m1, m1_err = photometric_mass(idx,save=True,plot=False,n_realizations=100)
            m1, m1_err = sources[sources['idx'] == idx]['m1'][0], sources[sources['idx'] == idx]['m1_err'][0]
            plot_mass_interp(age,mh,ebv,new_sources[new_sources['idx'] == idx]['bp_rp'],new_sources[new_sources['idx'] == idx]['mg'],m1,m1_err,idx,cluster,save,plot)
            m1_col.append(m1)
            m1_err_col.append(m1_err)
        except:
            print(f'Error in photometric mass for candidate {idx}')
            m1_col.append(np.nan)
            m1_err_col.append(np.nan)
        # plot_cmd(idx,cluster,age,mh,ebv,save,plot)

In [24]:
new_sources['m1'] = m1_col
new_sources['m1_err'] = m1_err_col

# new_sources.write('../table_B.fits',overwrite=True)
sources = new_sources

# AMRF

In [8]:
# Auxiliary functions, gaia passbands

# ------------ AMRF limits ----------------
from astropy.io import ascii
from synphot import SourceSpectrum, ReddeningLaw
from synphot.models import BlackBodyNorm1D
from synphot.units import convert_flux
from astropy import constants as const
from astropy import units as u
import json 
from uncertainties import unumpy as unp, ufloat
from uncertainties import correlated_values_norm, correlation_matrix
import warnings

def blackbody(temperature, wavelength, ebv=None, extinction_model='mwavg'):
    bb = SourceSpectrum(BlackBodyNorm1D, temperature=temperature*u.K)  # [photons s^-1 cm^-2 A^-1]
    # if ebv is not None:
    #     # apply extinction
    #     ext = ReddeningLaw.from_extinction_model(extinction_model).extinction_curve(ebv)
    #     bb = bb * ext
    bb = bb(wavelength)/(const.R_sun / const.kpc) ** 2  # undo synphot normalization (but leave the pi factor from integration over half a sphere)
    bb = convert_flux(wavelength, bb, 'flam')  # [flam] = [erg s^-1 cm^-2 A^-1]
    bb = bb.to(u.erg/u.s/u.cm**2/u.angstrom)  # express in normal astropy units
    return bb


def mlogg2radius(m, logg):
    g = 10**logg*u.cm/u.s**2
    r = np.sqrt(const.G*m/g)
    return r.to(u.Rsun).value


def calc_synth_phot(wavelength, flux, bandpass):
    dlambda = np.diff(wavelength)
    dlambda = np.concatenate([dlambda, np.array([dlambda[-1]])])

    # assuming a photon-counting device
    phot = np.sum(dlambda*bandpass*wavelength*flux)/np.sum(dlambda*bandpass*wavelength)
    
    return phot


amrf = lambda q, S : q/(1+q)**(2/3)*(1 - S*(1+q)/(q*(1+S)))

gaia_passband = ascii.read('../data/other/passband.dat', names=["wl", "gPb", "gPbError", "bpPb", "bpPbError", "rpPb", "rpPbError"])

# replace missing values with NaNs
for col in gaia_passband.itercols():
    col[col == 99.99] = 0
    
gaia_passband['wl'] *= 10  # [A]

# ---------- AMRF -------------------

limiting_curves = Table.read('../data/other/AMRF_limiting_curves.fits')
# Retrieve the conservative limiting AMRF values for some primary mass
# --------------
def Atr(m1):
    j = np.argmin(np.abs(m1 - limiting_curves['m1'].data))
    return limiting_curves['Atr'][j]
def Ams(m1):
    j = np.argmin(np.abs(m1 - limiting_curves['m1'].data))
    return limiting_curves['Ams'][j]

# =============================================================================
#                Auxil routines to obtain the covariance matrix
# =============================================================================

# 1) get the list of parameters from the solution type
def get_par_list(solution_type=None):
    if (solution_type is None) or (solution_type=='Orbital'):
        return ('ra', 'dec', 'parallax', 'pmra', 'pmdec', 'a_thiele_innes',
                'b_thiele_innes', 'f_thiele_innes', 'g_thiele_innes',
                'eccentricity', 'period', 't_periastron')

    elif (solution_type=='OrbitalAlternative') or (solution_type=='OrbitalAlternativeValidated') \
            or (solution_type=='OrbitalTargetedSearch') or (solution_type=='OrbitalTargetedSearchValidated'):
        return ('ra', 'dec', 'parallax', 'pmra', 'pmdec', 'a_thiele_innes',
                'b_thiele_innes', 'f_thiele_innes', 'g_thiele_innes',
                'period', 'eccentricity', 't_periastron')

    elif solution_type=='AstroSpectroSB1':
        return ('ra', 'dec', 'parallax', 'pmra', 'pmdec', 'a_thiele_innes',
                'b_thiele_innes', 'f_thiele_innes', 'g_thiele_innes',
                'c_thiele_innes', 'h_thiele_innes', 'center_of_mass_velocity',
                'eccentricity', 'period', 't_periastron')


# 2) Get the order of parameters in the covariance matrix, for a given bit index.
def bit_index_map(bit_index):
    if bit_index==8191:
        return ['ra','dec','parallax','pmra','pmdec','A','B','F','G', 'e','P', 'T']
    elif bit_index==8179:
        return ['ra','dec','parallax','pmra','pmdec','A','B','F','P', 'T']
    elif bit_index==65535:
        return ['ra', 'dec', 'parallax', 'pmra', 'pmdec', 'A', 'B', 'F', 'G', 'C', 'H', 'gamma','e', 'P', 'T']
    elif bit_index==65435:
        return ['ra', 'dec', 'parallax', 'pmra', 'pmdec', 'A', 'B', 'F', 'H', 'gamma', 'P', 'T']
    else:
        return None
    

# 3) Generate the correlation matrix
def make_corr_matrix(input_table, pars=None):
    """
    INPUT:
    input_table nss_two_body_orbit table.
    pars : list
            list of parameters for the corresponding solution of the desired
              target, in the same order as they appear in the Gaia table.
      """
    if pars is None:
        pars = get_par_list()

    # read the correlation vector
    s1 = input_table['corr_vec'].replace('\n','')   
    s1 = s1.replace(' ',',')
    s1 = s1.replace('--','0')
    corr_vec = list(json.loads(s1))
    # set the number of parameters in the table
    n_pars = len(pars)
    # define the correlation matrix.
    corr_mat = np.ones([n_pars, n_pars], dtype=float)

    # Read the matrix (lower triangle)
    ind = 0
    for i in range(n_pars):
        for j in range(i):
            corr_mat[j][i] = corr_vec[ind]
            corr_mat[i][j] = corr_vec[ind]
            ind += 1

    return corr_mat


# 4) Get the NSS data 
def get_nss_data(input_table, source_id):

    target_idx = np.argwhere(input_table['source_id'] == source_id)[0][0]
    pars = get_par_list(input_table['nss_solution_type'][target_idx])
    corr_mat = make_corr_matrix(input_table[target_idx], pars=pars)

    mu, std = np.zeros(len(pars)), np.zeros(len(pars))
    for i, par in enumerate(pars):
        try:
            mu[i] = input_table[par][target_idx]
            std[i] = input_table[par + '_error'][target_idx]
        except KeyError:
            mu[i], std[i] = np.nan, np.nan

    nan_idxs = np.argwhere(np.isnan(corr_mat))
    corr_mat[nan_idxs[:, 0], nan_idxs[:, 1]] = 0.0

    return mu, std, corr_mat

def multivar_sample(mu, sigma, corr, n):
    cov = corr*(sigma[:, None] * sigma[None, :])
    # l = spla.cholesky(cov)
    # z = np.random.normal(size=(n, mu.shape[0]))
    # return z.dot(l) + mu
    return np.random.multivariate_normal(mu, cov, size=n)

# =============================================================================
#                      calc parameters.
# =============================================================================
# Here we calculate the AMRF, qmin, etc, assuming that the mass of the luminous star 
# is exactly one solar mass. This is just for the red-clump stars...
def calc_AMRF(par_in, par_in_errors, corr_matrix, m1, bit_index=8191):
    """
    For the given set of orbital parameters by Gaia, this function calculates
    the standard geometrical elements (a, omega, Omega, and i). If the error
    estimates and covariance matrix are prodived, the error estimates on the
    calculated parameters are returned as well.

    Input: thiele_innes: Thiele Innes parameters [A,B,F,G] in milli-arcsec
           thiele_innes_errors : Corresponding errors.
           corr_matrix : Corresponding  4X4 correlation matrix.

  Output: class-III probability via monte carlo
    """
    # Read the coefficients and assign the correlation matrix.
    # Create correlated quantities. If the error is nan we assume 1e-6...
    par_in_errors[np.isnan(par_in_errors)] = 1e-6
    par_list = correlated_values_norm([(par_in[i], par_in_errors[i]) for i in np.arange(len(par_in))], corr_matrix)
    key_list = bit_index_map(bit_index)

    par = {key_list[i]: par_list[i] for i in np.arange(len(key_list))}
    par['mass'] = m1

    # Add the G Thiele-Innes parameter if needed.
    if (bit_index == 8179) | (bit_index == 65435):
        G = -par['A']*par['F']/par['B']
    else:
        G = par['G']

    # This in an intermediate step in the formulae...
    p = (par['A'] ** 2 + par['B'] ** 2 + G ** 2 + par['F'] ** 2) / 2.
    q = par['A'] * G - par['B'] * par['F']

    # Calculate the angular semimajor axis (already in mas)
    a_mas = unp.sqrt(p + unp.sqrt(p ** 2 - q ** 2))

    # Calculate the inclination and convert from radians to degrees
    i_deg = unp.arccos(q / (a_mas ** 2.)) * (180 / np.pi)
    
    try:
        if par.get("d") is not None:
            K_kms = 4.74372*unp.sqrt(par['C'] ** 2 + par['H'] ** 2)*(2*np.pi)/(par['P']/ 365.25)/np.sqrt(1-par['e']**2)
            acc   = 2*K_kms/par['P']
        else:
            K_kms = 4.74372*unp.sqrt(par['C'] ** 2 + par['H'] ** 2)*(2*np.pi)/(par['P']/ 365.25)
            acc   = K_kms/par['P']/4
    except:
        K_kms = ufloat(999, 999) 
        acc   = ufloat(999,999)

    # Calculate the AMRF
    try:
        AMRF = a_mas / par['parallax'] * par['mass'] ** (-1 / 3)  * (par['P']/ 365.25) ** (-2 / 3)

        # Calculate AMRF q
        y = AMRF ** 3
        h = (y/2 + (y**2)/3 + (y**3)/27
             + np.sqrt(3)/18*y*unp.sqrt(4*y+27))**(1/3)
        q = h + (2*y/3 + (y**2)/9)/h + y/3
    except:
        AMRF = ufloat(np.nan, np.inf) 
        q    = ufloat(np.nan, np.inf) 
        
    # Extract expectancy values and standard deviations
    pars = np.array([unp.nominal_values(AMRF),
                         unp.nominal_values(q),
                         unp.nominal_values(a_mas),
                         unp.nominal_values(i_deg),
                         unp.nominal_values(K_kms),
                         unp.nominal_values(acc)])

    pars_error = np.array([unp.std_devs(AMRF),
                               unp.std_devs(q),
                               unp.std_devs(a_mas),
                               unp.std_devs(i_deg),
                               unp.std_devs(K_kms),
                               unp.std_devs(acc)])

    return pars, pars_error

def class_probs(Atr,Ams,par_in, par_in_errors,
                  m1, m1_error, corr_matrix, bit_index=8191, n=1e2, factor=1.0):
    """
    For the given set of orbital parameters by Gaia, this function calculates
    the standard geometrical elements (a, omega, Omega, and i). If the error
    estimates and covariance matrix are prodived, the error estimates on the
    calculated parameters are returned as well.

    Input: thiele_innes: Thiele Innes parameters [A,B,F,G] in milli-arcsec
           thiele_innes_errors : Corresponding errors.
           corr_matrix : Corresponding  4X4 correlation matrix.

  Output: physical and geometrical parameters
    """
    r_3 = 0
    r_2 = 0
    par_in_errors[np.isnan(par_in_errors)] = 1e-6
    vecs = multivar_sample(par_in, par_in_errors, corr_matrix, int(n))
    key_list = bit_index_map(bit_index)

    for vec in vecs:
        par = {key_list[i]: vec[i] for i in np.arange(len(key_list))}
        par['mass'] = m1_error*np.random.randn() + m1

        # Add the G Thiele-Innes parameter if needed.
        if (bit_index == 8179) | (bit_index == 65435):
            par['G'] = -par['A'] * par['F'] / par['B']

        # This in an intermediate step in the formulae...
        p = (par['A'] ** 2 + par['B'] ** 2 + par['G'] ** 2 + par['F'] ** 2) / 2.
        q = par['A'] * par['G'] - par['B'] * par['F']

        # Calculate the semimajor axis (already in mas)
        a_mas = np.sqrt(p + np.sqrt(p ** 2 - q ** 2))

        # Calculate the AMRF
        AMRF = a_mas / par['parallax'] * par['mass'] ** (-1 / 3) * (par['P']/ 365.25) ** (-2 / 3)

        try:
            if 0 < par['e'] < 1:
                if AMRF > Atr * factor:
                    r_3 += 1
                elif Ams * factor < AMRF < Atr * factor:
                    r_2 += 1
        except KeyError:
            pass

    return (n-r_2-r_3)/n, r_2/n, r_3/n #(no_detections + detections)

# =============================================================================
#                       Read the data from the NSS table
# =============================================================================
def add_astrometric_parameters(data):
    # Here we only calculate (but don't assign class 3 probabilities!
    # We get the data table, arrange the arrays, calculate the astrometric
    # coefficients and plug it all back into the table.

    # Initialize the arrays
    # ---------------------
    # We need to calculate the AMRF, mass ratio, angular semi-major axis, orbtial inclination
    # and order-of-magnitude acceleration. We also want their uncertainties.
    count_good, count_bad = 0, 0
    A, q, a_mas, i_deg, K_kms, acc, P1, P2, P3 = np.full(len(data), np.nan),  np.full(len(data), np.nan), \
                              np.full(len(data), np.nan),  np.full(len(data), np.nan), \
                              np.full(len(data), np.nan), np.full(len(data), np.nan), \
                              np.full(len(data), np.nan), np.full(len(data), np.nan), np.full(len(data), np.nan) 

    Ae, qe, a_mase, i_dege, K_kmse, acce = np.full(len(data), np.nan),  np.full(len(data), np.nan), \
                                   np.full(len(data), np.nan),  np.full(len(data), np.nan), \
                                   np.full(len(data), np.nan),  np.full(len(data), np.nan)

    # Now go one by one and calculate the AMRF
    # ----------------------------------------
    for idx in tqdm(range(len(data['source_id']))):
        if data[idx]['nss_solution_type'] not in ['Orbital','AstroSpectroSB1']:
            continue
        # Read the NSS solutin values.
        sid = data['source_id'][idx]
        mu, std, corr_mat = get_nss_data(data, sid)
        m1 = data['m1'][idx]
        m1_error = data['m1_err'][idx]
        if np.ma.is_masked(m1):
            print(idx)
            pass
        Ams_idx = data['Ams'][idx]
        Atr_idx = data['Atr'][idx]
        if np.ma.is_masked(Ams_idx) | np.ma.is_masked(Atr_idx):
            Ams_idx = Ams(m1)
            Atr_idx = Atr(m1)
            
        vals, stds = calc_AMRF(mu, std, corr_mat, m1, bit_index=data['bit_index'][idx])
        p1, p2, p3 = class_probs(data['Atr'][idx],data['Ams'][idx],mu, std, m1, m1_error, corr_mat, bit_index=data['bit_index'][idx], n = 1e4)
        try:
            A[idx], Ae[idx]  = vals[0], stds[0]
            q[idx], qe[idx]  = vals[1], stds[1]
            a_mas[idx], a_mase[idx]  = vals[2], stds[2]
            i_deg[idx], i_dege[idx]  = vals[3], stds[3]
            K_kms[idx], K_kmse[idx]  = vals[4], stds[4]
            acc[idx],   acce[idx]    = vals[5], stds[5]
            P1[idx], P2[idx], P3[idx] = p1, p2, p3
        except:
            pass
    
    # Store it all back in the original data structure.
    data['AMRF'], data['AMRF_error'] = A, Ae
    data['AMRF_q'], data['AMRF_q_error'] = q, qe
    data['a_mas'], data['a_mas_error'] = a_mas, a_mase
    data['i_deg'], data['i_deg_error'] = i_deg, i_dege
    data['K_kms'], data['K_kms_error'] = K_kms, K_kmse
    data['acc_kmsd'], data['acc_kmsd_error'] = acc, acce
    data['classI_prob'] = P1
    data['classII_prob'] = P2
    data['classIII_prob'] = P3
    return data


In [9]:
# Calc AMRF limits

# sources['Ams'] = np.full(len(sources),np.nan)
# sources['Atr'] = np.full(len(sources),np.nan)

## calculating for all sources

stage_min = 0  # pre-main sequence
stage_max = 5  # red giant branch
mass_min = 0  # [Msun]
mass_max = 8  # [Msun]

m2_vec = np.arange(0.1, 10, 0.1)

wavelength = gaia_passband['wl'].value  # [A]

Gflux1 = np.zeros(len(sources))
Gflux2 = np.zeros((len(sources), len(m2_vec)))
Ag1 = np.zeros(len(sources))
Ag2 = np.zeros((len(sources), len(m2_vec)))
q = np.zeros((len(sources), len(m2_vec)))
# PARSEC_path = '../data/PARSEC v1.2S/Gaia_lin/'
PARSEC_path = '../OCFit/gaiaDR2/grids/'
models = stam.getmodels.read_parsec(path=PARSEC_path)
for i in tqdm(range(len(sources))):
    idx = sources['idx'][i]
    # only for candidate 110. for rest, skip
    if idx != 110:
        continue
    if sources[i]['nss_solution_type'] not in ['Orbital','AstroSpectroSB1']:
        continue
    mh = sources['[Fe/H]'][i]
    age = sources['age'][i]
    ebv = sources['Av'][i]/3.1
    m1 = sources['m1'][i]
    
    try:
        track_idx = get_track_idx(mh,age)[-1]
        tracks = models[track_idx].copy()
        tracks.sort('Mini')
        idx = np.argmin(np.abs(tracks['Mass'] - m1))

        teff1 = 10**tracks['logTe'][idx]  # [K]
        logg1 = tracks['logg'][idx]
        r1 = mlogg2radius(m1*u.Msun, logg1)  # [Rsun]
        flux1 = blackbody(teff1, wavelength)*4*np.pi*r1**2
        Gflux1[i] = calc_synth_phot(wavelength, flux1, gaia_passband['gPb'].value).value
        Ag1[i],_ = extinction.get_AG_EBPRP(3.1*ebv,0,teff1)
        q[i, :] = m2_vec/m1  # mass ratio

        for j in range(len(m2_vec)):
            m2 = m2_vec[j]
            idx = np.argmin(np.abs(tracks['Mass'] - m2))
            teff2 = 10**tracks['logTe'][idx]  # [K]
            logg2 = tracks['logg'][idx]
            r2 = mlogg2radius(m2*u.Msun, logg2)  # [Rsun]
            flux2 = blackbody(teff2, wavelength, ebv=ebv)*4*np.pi*r2**2
            Gflux2[i,j] = calc_synth_phot(wavelength, flux2, gaia_passband['gPb'].value).value
            Ag2[i,j],_ = extinction.get_AG_EBPRP(3.1*ebv,0,teff2)

        Sms = Gflux2[i,:]/Gflux1[i]*10**(-0.4*(Ag2[i,:]-Ag1[i]))
        Ams = amrf(q[i, :], Sms)
        valid_idx = Sms < 1
        Ams = np.max(Ams[valid_idx]) 
        sources['Ams'][i] = Ams
        Str = 2*Gflux2[i,:]/Gflux1[i]
        Atr = amrf(2*q[i, :], Str)
        valid_idx = Str < 1
        Atr = np.max(Atr[valid_idx])
        sources['Atr'][i] = Atr
    except:
        print(f'i = {i}')

100%|██████████| 37/37 [00:01<00:00, 22.80it/s]


In [11]:
# Calc AMRF and class probabilities
new_sources = add_astrometric_parameters(sources)
new_sources['m2'] = new_sources['m1'] * new_sources['AMRF_q']
new_sources['m2_err'] = ((new_sources['m1_err'] * new_sources['AMRF_q'])**2 + (new_sources['m1'] * new_sources['AMRF_q_error'])**2)**(1/2)

100%|██████████| 37/37 [00:03<00:00, 11.88it/s]


In [13]:
sources = new_sources
cut1 = sources['significance'] > 158 * sources['period'].data**(-0.5)
cut2 = (sources['parallax_over_error'] > 20,000 *sources['period']**(-1))[0]
cut3 = sources['eccentricity_error'] < 0.079 * np.log(sources['period'].data) - 0.244

cut = cut1 & cut2 & cut3

sources = sources[cut]

# sources.write('../table_B.fits',overwrite=True)

In [16]:
cut1 = sources['m1'] / sources['m1_err'] > 50
cut2 = sources['classI_prob'] < 0.1
cut3 = sources['m2'] <= 1.4

cut = cut1 & cut2 & cut3

sources[cut].write('../table_C.fits',overwrite=True)

In [21]:
import shutil
tbl = Table.read('../table_C.fits')

for i in range(len(tbl)):
    idx = tbl['idx'][i]
    cluster = tbl['cluster'][i]
    # cmd_fig_name = f'../img/cmd/{idx}_{cluster}.png'
    # cmd_new_name = f'../img/cmd_tableC/{idx}_{cluster}.png'
    # shutil.move(cmd_fig_name, cmd_new_name)
    interp_fig_name = f'../img/mass_interp/{idx}_{cluster}.png'
    interp_new_name = f'../img/mass_interpC/{idx}_{cluster}.png'
    shutil.move(interp_fig_name, interp_new_name)


# Calculating IFMR, Kroupa distributions etc.

In [2]:
## t_life vs m from evolutionary tracks/M_i from t_cluster, t_cool
from os import walk,path

def get_zarr(): # metallicities of parsec evol tracks
    basedir = path.join('..','data','PARSEC v1.2S','evol_tracks')
    filenames = walk(basedir).__next__()[2]
    z_arr = []
    for n in filenames:
        z = n.split('Y')[0]
        z_arr.append(float(z[1:]))
    return np.unique(z_arr)

def get_mdict(mh): # masses of evol tracks for a given metallicity
    basedir = path.join('..','data','PARSEC v1.2S','evol_tracks')
    filenames = walk(basedir).__next__()[2]
    z_arr = get_zarr()
    z = 0.01524 * 10**mh
    z_nearest = z_arr[np.argmin(np.abs(z_arr - z))]
    z_str = 'Z'+f'{z_nearest}'

    m_arr = []
    s_arr = []
    for n in filenames:
        if z_str in n:
            m = n.split('M')[1].removesuffix('.DAT')
            if m.endswith('.HB'):
                m = m.removesuffix('.HB')
            m_arr.append(float(m))
            s_arr.append(m)
    tbl = Table({'mass':m_arr,'str':s_arr})
    # tbl = tbl[tbl['mass'] <= 10]
    return tbl

def get_marr(mh): # masses of evol tracks for a given metallicity
    return np.unique(get_mdict(mh)['mass'])

def get_lifetime(mass,mh): # MS lifetime for given mass (>0.8) and metallicity
    ## lifetime for m>0.8
    basedir = path.join('..','data','PARSEC v1.2S','evol_tracks')
    filenames = walk(basedir).__next__()[2]
    z_arr = get_zarr()

    z = 0.01524 * 10**mh
    z_nearest = z_arr[np.argmin(np.abs(z_arr - z))]
    m_dict = get_mdict(mh)
    mass_str = m_dict[m_dict['mass'] == mass]['str']
    z_str = 'Z'+f'{z_nearest}'

    trackpath = ''
    for n in filenames:
        m1 = n.split('M')[1].removesuffix('.DAT')
        if m1.endswith('HB'):
            continue
        for m2 in mass_str:
            if m1 == m2 and z_str in n:
                trackpath = path.join(basedir,n)
                break

    if trackpath == '':
        print(f'No track found for '+m2+' '+z_str)

    trck = Table(np.genfromtxt(trackpath,names=True,dtype=None))
    # return trck[trck['PHASE'] == 6]['AGE'][0] ## end of MS
    if len(trck[trck['PHASE'] == 14]['AGE']) == 0:
        return trck[trck['PHASE'] == 11]['AGE'][0] ## TRGB
    return trck[trck['PHASE'] == 14]['AGE'][0] ## start of AGB

def lifetime_vs_mass(mh): # MS lifetime for different masses (>0.8), fixed metallicity
    m_arr = get_marr(mh)
    m_arr = m_arr[m_arr > 0.8]
    t_arr = []
    for m in m_arr:
        t_arr.append(get_lifetime(m,mh))
    return m_arr,t_arr

def create_lifetime_vs_mass_tables(): # create csv files for MS lifetime vs mass for different metallicities
    z_arr = get_zarr()
    mh_arr = np.log10(z_arr/0.01524)

    for mh in mh_arr:
        m_arr,t_arr = lifetime_vs_mass(mh)
        tbl = Table({'mass':m_arr,'lifetime':t_arr})
        tbl.write(path.join('..','data','MS_lifetime','parsec_V1.2S',f'lifetime_vs_mass{mh:.2f}.csv'),overwrite=True)

    return ''

def get_initial_mass(age,mh,phase='AGB'): # from MS lifetime and metallicity, get progenitor mass (invert get_lifetime)
    ## age in Myr
    ## phase either MSTO (main sequence turnoff) or AGB (start of asymptotic giant branch, helium depletion in core)
    ## For a given lifetime, AGB gives larger mass than MSTO (more evolved). As lifetime increases, this difference becomes negligible.
    ## returns mass in solar mass
    basedir = path.join('..','data','MS_lifetime',phase)
    z_arr = get_zarr()
    mh_arr = np.log10(z_arr/0.01524)
    mh = mh_arr[np.argmin(np.abs(mh_arr - mh))]
    filename = f'lifetime_vs_mass{mh:.2f}.csv'   
    df = pd.read_csv(path.join(basedir,filename))
    df.sort_values('lifetime',inplace=True)
    return np.interp(age*1e6,df['lifetime'],df['mass'])
    
def get_cooling_age(teff,m,core='CO',atm='H'): # WD cooling age for given teff, mass, core, atm
    if core =='He':
        model = np.genfromtxt('../data/WD_models/He_wd.dat',names=True,dtype=None)
    elif core == 'CO':
        if atm == 'H':
            model = np.genfromtxt('../data/WD_models/CO_DA.dat',names=True,dtype=None)
        elif atm == 'He':
            model = np.genfromtxt('../data/WD_models/CO_DB.dat',names=True,dtype=None)
        else:
            print('atm either H or He')
            return None
    elif core == 'ONe':
        if atm == 'H':
            model = np.genfromtxt('../data/WD_models/ONe_DA.dat',names=True,dtype=None)
        elif atm == 'He':
            model = np.genfromtxt('../data/WD_models/ONe_DB.dat',names=True,dtype=None)
        else:
            print('atm either H or He')
            return None
    # logg = logg_from_MR_relation(teff,m,core,atm)
    interp = LinearNDInterpolator(np.array([model['Teff'],model['Mass']]).T,model['Age'])
    age = interp([teff,m])[0]
    if np.isnan(age):
        return np.nan
    return age

def get_cooling_temp(age,m,core='CO',atm='H'): # WD cooling temp for given age, mass, core, atm
    if core =='He':
        model = np.genfromtxt('../data/WD_models/He_wd.dat',names=True,dtype=None)
    elif core == 'CO':
        if atm == 'H':
            model = np.genfromtxt('../data/WD_models/CO_DA.dat',names=True,dtype=None)
        elif atm == 'He':
            model = np.genfromtxt('../data/WD_models/CO_DB.dat',names=True,dtype=None)
        else:
            print('atm either H or He')
            return None
    elif core == 'ONe':
        if atm == 'H':
            model = np.genfromtxt('../data/WD_models/ONe_DA.dat',names=True,dtype=None)
        elif atm == 'He':
            model = np.genfromtxt('../data/WD_models/ONe_DB.dat',names=True,dtype=None)
        else:
            print('atm either H or He')
            return None
    interp = LinearNDInterpolator(np.array([model['Age'],model['Mass']]).T,model['Teff'])
    teff_init = interp([age,m])[0]
    teff_vec = np.linspace(teff_init+10000,teff_init-10000,20)
    age_vec = [get_cooling_age(teff,m,core,atm) for teff in teff_vec]
    teff = np.interp(age,age_vec,teff_vec)
    return teff

def logg_from_MR_relation(teff,m,core='CO',atm='H'):
    ## teff in K, m in solar masses
    ## returns log_g in cm/s^2
    if core == 'He':
        wd = np.genfromtxt('../data/WD_models/He_wd.dat',names=True,dtype=None)
    elif core == 'CO':
        if atm == 'H':
            wd = np.genfromtxt('../data/WD_models/CO_DA.dat',names=True,dtype=None)
        elif atm == 'He':
            wd = np.genfromtxt('../data/WD_models/CO_DB.dat',names=True,dtype=None)
        else:
            print('atm either H or He')
            return None
    elif core == 'ONe':
        if atm == 'H':
            wd = np.genfromtxt('../data/WD_models/ONe_DA.dat',names=True,dtype=None)
        elif atm == 'He':
            wd = np.genfromtxt('../data/WD_models/ONe_DB.dat',names=True,dtype=None)
        else:
            print('atm either H or He')
            return None
    interp = LinearNDInterpolator(np.array([wd['Teff'],wd['Mass']]).T,wd['log_g'])
    logg = interp([teff,m])[0]
    return logg

def logg_from_agecool_m(age,m,core='CO',atm='H'):
    ## teff in K, m in solar masses
    ## returns log_g in cm/s^2
    if core == 'He':
        wd = np.genfromtxt('../data/WD_models/He_wd.dat',names=True,dtype=None)
    elif core == 'CO':
        if atm == 'H':
            wd = np.genfromtxt('../data/WD_models/CO_DA.dat',names=True,dtype=None)
        elif atm == 'He':
            wd = np.genfromtxt('../data/WD_models/CO_DB.dat',names=True,dtype=None)
        else:
            print('atm either H or He')
            return None
    elif core == 'ONe':
        if atm == 'H':
            wd = np.genfromtxt('../data/WD_models/ONe_DA.dat',names=True,dtype=None)
        elif atm == 'He':
            wd = np.genfromtxt('../data/WD_models/ONe_DB.dat',names=True,dtype=None)
        else:
            print('atm either H or He')
            return None
    interp = LinearNDInterpolator(np.array([wd['Age'],wd['Mass']]).T,wd['log_g'])
    logg = interp([age,m])[0]
    return logg

def get_temp_from_mag(m,mag,parallax,av,band='NUV',atm='H'): # for apparent UV mag/WD mass/parallax/av, get WD temp
    if band not in ['NUV','FUV']:
        print('band must be either NUV or FUV')
        return None
    if atm == 'H':
        model = np.genfromtxt(f'../data/WD_models/CO_DA.dat',names=True,dtype=None)
    elif atm == 'He':
        model = np.genfromtxt(f'../data/WD_models/CO_DB.dat',names=True,dtype=None)
    interp = LinearNDInterpolator(np.array([model['Mass'],model[band]]).T,model['Teff'])
    abs_mag = mag - 5*np.log10(1000/parallax) + 5
    teff_init = interp([m,abs_mag])[0] # rough temperature for dereddening the observed mag
    if np.isnan(teff_init):
        interp = NearestNDInterpolator(np.array([model['Mass'],model[band]]).T,model['Teff'])
        teff_init = interp([m,abs_mag])[0] # rough temperature for dereddening the observed mag
    ANUV,_ = extinction.get_Galex_extinction(av,0,teff_init)
    abs_mag = abs_mag - ANUV # dereddened mag
    teff = interp([m,abs_mag])[0] # final temperature
    if np.isnan(teff):
        interp = NearestNDInterpolator(np.array([model['Mass'],model[band]]).T,model['Teff'])
        teff = interp([m,abs_mag])[0]
    return int(teff)

def add_temperature_limits_to_table(tbl): # Use cluster age and observed UV to constrain teff2

    tbl['teff2_max'] = np.full(len(tbl),np.nan)
    tbl['teff2_min'] = np.full(len(tbl),np.nan)

    for idx in tbl['idx']:
        j = np.argwhere(tbl['idx'] == idx)[0][0]
        age = tbl['age'][j]
        av = tbl['Av'][j]
        mh = tbl['[Fe/H]'][j]
        parallax = tbl['parallax'][j]
        m2 = tbl['m2'][j]
        nuv_mag = tbl['nuv_mag'][j]
        fuv_mag = tbl['fuv_mag'][j]
        
        if m2<0.45: core = 'He'
        elif m2<1.1: core = 'CO'
        else: core = 'ONe'

        # if UV photometry exists, UV excess can give strict limits on Teff2
        # if no UV excess, we can still put upper bound on Teff2: the highest possible Teff2 must provide less than 10% of the observed UV flux, or else there'd be excess
        if not np.ma.is_masked(fuv_mag): 
            tbl[j]['teff2_max'] = get_temp_from_mag(m2,fuv_mag+2.5,parallax,av,band='FUV',atm='H')
        elif not np.ma.is_masked(nuv_mag): 
            tbl[j]['teff2_max'] = get_temp_from_mag(m2,nuv_mag+2.5,parallax,av,band='NUV',atm='H')
        tbl[j]['teff2_min'] = get_cooling_temp(age,m2,core,'H')

    tbl['teff2'] = np.full(len(tbl),np.nan)
    excess = [53,174,175,236,249,281,283]
    for idx in excess:
        j = np.argwhere(tbl['idx'] == idx)[0][0]
        t_best, t_min, t_max = get_fit_results(idx)
        tbl[j]['teff2'] = t_best
        tbl[j]['teff2_min'] = t_min
        tbl[j]['teff2_max'] = t_max

def get_fit_results(idx): # get median and 16th, 84th percentiles of teff2 chain
    filepath = f'../data/chains/chains_{idx}.csv'
    chain = Table.read(filepath)['teff2']
    return np.median(chain),np.percentile(chain,16),np.percentile(chain,84)

def sample_teff2_posterior(idx,n=1e4,tclstr=None): # sample teff2 from posterior using rejection sampling
    filepath = f'../data/chains/chains_{idx}.csv'
    chain = Table.read(filepath)['teff2']
    counts,bins = np.histogram(chain,bins=100)
    tmax = bins[-1]
    tmin = bins[0]
    teff2 = []
    if tclstr is not None:
        tmin = np.max([tmin,tclstr])
    while len(teff2) < n:
        t = np.random.uniform(tmin,tmax)
        if np.random.uniform(0,1) < np.interp(t,bins[:-1],counts)/np.max(counts):
            teff2.append(t)
    return np.array(teff2)

def sample_m_init_posterior(tbl,idx,n=1e4): # sample m_init from teff2 posterior, age, mh, m2 
    j = np.argwhere(tbl['idx'] == idx)[0][0]
    age, e_age = tbl[j]['age'],tbl[j]['e_age']
    mh, e_mh = tbl[j]['[Fe/H]'],tbl[j]['e_[Fe/H]']
    m2, m2_err = tbl[j]['m2'],tbl[j]['m2_err']

    atm = 'H'
    if m2<0.45: core = 'He'
    elif m2<1.1: core = 'CO'
    else: core = 'ONe'

    # tclstr = get_cooling_temp(age,m2,core,atm)
    teff2_vec = sample_teff2_posterior(idx,n=10*n)
    age_vec = []
    mh_vec = []
    m2_vec = []
    t2_vec = []
    m_i_vec = []
    k = 0
    with tqdm(total=int(n)) as pbar:
        while (np.count_nonzero(~np.isnan(m_i_vec) & (np.array(m_i_vec) < 8)) < n):
            age_vec.append(np.random.normal(age,e_age))
            mh_vec.append(np.random.normal(mh,e_mh))
            m2_vec.append(np.random.normal(m2,m2_err))

            # t_ms_min = get_lifetime(8.0,mh_vec[-1])
            # age_wd_max = age_vec[-1] - t_ms_min
        
            t2_vec.append(np.random.choice(teff2_vec))

            age_wd = get_cooling_age(t2_vec[-1],m2_vec[-1],core,atm)
            t_ms = age_vec[-1] - age_wd
            if t_ms > 0:
                m_i = get_initial_mass(t_ms,mh_vec[-1],phase='AGB')
                m_i_vec.append(m_i)
                if not np.isnan(m_i) and m_i < 8:
                    pbar.update(1)
            else:
                m_i_vec.append(np.nan)
    return Table({'m_i':m_i_vec,'age':age_vec,'[Fe/H]':mh_vec,'m2':m2_vec,'teff2':t2_vec})

def weight_m_init_posterior(idx,n=1e4): # weight m_init chain using kroupa IMF
    filepath = f'../data/chains/mchain_{idx}.csv'
    chain_tbl = Table.read(filepath)
    m_chain = chain_tbl['m_i']
    age_chain = chain_tbl['age']
    mh_chain = chain_tbl['[Fe/H]']
    m2_chain = chain_tbl['m2']
    teff2_chain = chain_tbl['teff2']

    cut = ~np.isnan(m_chain) & (m_chain < 8)
    m_chain = m_chain[cut]
    age_chain = age_chain[cut]
    mh_chain = mh_chain[cut]

    n = len(m_chain)
    m_chain_w, age_chain_w, mh_chain_w, m2_chain_w, teff2_chain_w = [],[],[],[],[]


    kroupa = lambda m,m0: (m/m0)**(-2.3) # peak normalized to 1
    while len(m_chain_w) < n:
        i = np.random.randint(0,n)
        m = m_chain[i]
        age = age_chain[i]
        mh = mh_chain[i]
        m2 = m2_chain[i]
        teff2 = teff2_chain[i]

        m_min = get_initial_mass(age,mh,phase='MSTO') # to define kroupa lower cut-off.
        p = np.random.random()
        if p < kroupa(m,m_min) and m > m_min and m < 8: # rejection sampling on truncated kroupa IMF
            m_chain_w.append(m)
            age_chain_w.append(age)
            mh_chain_w.append(mh)
            m2_chain_w.append(m2)
            teff2_chain_w.append(teff2)

    return Table({'m_i':m_chain_w,'age':age_chain_w,'[Fe/H]':mh_chain_w,'m2':m2_chain_w,'teff2':teff2_chain_w})

def sample_kroupa_uniform(m_min,m_max,n=1e4): # sample m from kroupa IMF with no prior
    kroupa = lambda m,m0: (m/m0)**(-2.3) # peak normalized to 1
    m_arr = []
    while len(m_arr) < n:
        m = np.random.uniform(m_min,m_max)
        p = np.random.random()
        if p < kroupa(m,m_min) and m > m_min and m < m_max: # rejection sampling on truncated kroupa IMF
            m_arr.append(m)
    return np.array(m_arr)

def create_ifmr_tbl(tbl): # create m_i vs. m_f table, with all levels of bounds
    ifmr = Table({'idx':tbl['idx'],'source_id':tbl['source_id'],'m1':tbl['m1'],'m2':tbl['m2'], 'm2_err':tbl['m2_err'], 'age':tbl['age'],
                  '[Fe/H]':tbl['[Fe/H]'], 'e_[Fe/H]':tbl['e_[Fe/H]'] ,'period':tbl['period'],'period_error':tbl['period_error'],
                  'age':tbl['age'], 'e_age':tbl['e_age'], 'eccentricity':tbl['eccentricity'],'eccentricity_error':tbl['eccentricity_error'],
                  'm_i_min_abs':np.full(len(tbl),np.nan),'m_i_min1':np.full(len(tbl),np.nan),'m_i_1':np.full(len(tbl),np.nan),'m_i_max1':np.full(len(tbl),np.nan),
                  'm_i_min2':np.full(len(tbl),np.nan),'m_i_2':np.full(len(tbl),np.nan),'m_i_max2':np.full(len(tbl),np.nan),
                  'm_i_min3':np.full(len(tbl),np.nan),'m_i_3':np.full(len(tbl),np.nan),'m_i_max3':np.full(len(tbl),np.nan)})
    # add_temperature_limits_to_table(tbl)

    for j in tqdm(range(len(ifmr))):
        idx = tbl[j]['idx']
        age = tbl[j]['age']
        mh = tbl[j]['[Fe/H]']
        
        # Loose bounds on m_init from kroupa IMF
        filepath = f'../data/chains/kroupa_{ifmr[j]["idx"]}.npy'
        if path.exists(filepath): 
            kchain = np.load(filepath)
            kchain = kchain[kchain < 8]
            if len(kchain)>0:
                ifmr[j]['m_i_min_abs'] = np.percentile(kchain,1)
                ifmr[j]['m_i_min1'] = np.percentile(kchain,16)
                ifmr[j]['m_i_max1'] = np.percentile(kchain,84)
                ifmr[j]['m_i_1'] = np.percentile(kchain,50)
            else:
                print(f'No kroupa chain for {idx}')

        # If no UV excess, use maximal teff2 to narrow down m_init
        uv = ~(np.isnan(tbl['nuv_mag']) & np.isnan(tbl['fuv_mag']))
        uv_excess = np.isin(tbl['idx'],[53,174,175,236,249,281,283])
        no_excess = uv & ~uv_excess
        if idx in tbl[no_excess]['idx']: 
            filepath = f'../data/chains/trunc_kroupa_{idx}.npy'
            if path.exists(filepath): # should always exist if no UV excess
                tchain = np.load(filepath)
                if len(tchain) > 1e4:
                    ifmr[j]['m_i_min2'] = np.nanpercentile(tchain,16)
                    ifmr[j]['m_i_2'] = np.nanpercentile(tchain,50)
                    ifmr[j]['m_i_max2'] = np.nanpercentile(tchain,84)
                else:
                    ifmr[j]['m_i_2'] = -1 # flag problematic cases. In these cases the cluster age is too young for the results to be meaningful.
            


        if idx in tbl[uv_excess]['idx']: # Use MCMC bounds on teff2 to get tighter bound on m_init including a best value
            filepath = f'../data/chains/wmchain_{idx}.npy'
            if path.exists(filepath): # otherwise, leave it as nan
                wmchain = np.load(filepath)
                ifmr[j]['m_i_min3'] = np.percentile(wmchain,16)
                ifmr[j]['m_i_3'] = np.percentile(wmchain,50)
                ifmr[j]['m_i_max3'] = np.percentile(wmchain,84)
            else:
                ifmr[j]['m_i_3'] = -1 # flag problematic cases. In these cases the cluster age is too young for the results to be meaningful.
    return ifmr

def inverse_cummings_ifmr(m_final):
    """
    Calculate the initial mass (M_i) given the final mass (M_f) using 
    the inverse of the Cummings et al. (2018) IFMR.
    
    Parameters:
    m_final (float): The final mass of the white dwarf (in solar masses).
    
    Returns:
    float: The initial mass of the progenitor star (in solar masses).
    """
    if m_final < (0.080 * 0.83 + 0.489):  # Check if M_f is too low for this IFMR
        # raise ValueError(f"M_f {m_final:.2f} is outside the valid range for this IFMR.")
        return np.nan
    
    if m_final < (0.080 * 2.85 + 0.489):  # For range 0.83 <= M_i < 2.85
        m_initial = (m_final - 0.489) / 0.080
    
    elif m_final < (0.187 * 3.60 + 0.184):  # For range 2.85 <= M_i < 3.60
        m_initial = (m_final - 0.184) / 0.187
    
    elif m_final <= (0.107 * 7.20 + 0.471):  # For range 3.60 <= M_i <= 7.20
        m_initial = (m_final - 0.471) / 0.107
    
    else:  # Check if M_f is too high for this IFMR
        # raise ValueError(f"M_f {m_final:.2f} is outside the valid range for this IFMR.")
        return np.nan
    
    return m_initial

def get_cunningham_ifmr_breakpoints():
    """
    Return the breakpoints and endpoints from Table 2 of the Cunningham 2024 paper,
    including M_initial, M_final, and their 1-sigma confidence intervals, sorted by M_initial
    in an Astropy Table.
    
    Returns:
        astropy.table.Table: A table containing M_initial, M_final, and their 1-sigma confidence intervals
                             taken directly from Table 2 of the Cunningham 2024 paper.
    """
   # Data and column names passed as a dictionary
    table2_data = {
        'M_initial': [1.09, 2.65, 3.42, 5.06, 7.44],
        'M_final': [0.561, 0.70, 0.79, 0.91, 1.3],
        'M_final_p1sig': [0.559, 0.68, 0.77, 0.88, 1.25],
        'M_final_m1sig': [0.563, 0.72, 0.81, 0.94, 1.35],
        'M_final_p2sig': [0.557, 0.66, 0.75, 0.85, 1.20],
        'M_final_m2sig': [0.565, 0.74, 0.83, 0.97, 1.40],
    }

    # Convert the dictionary to an Astropy Table
    table2 = Table(table2_data)

    # Sort the table by M_initial
    table2.sort('M_initial')

    return table2


In [ ]:
## create chains

sources = Table.read('../table_C.fits')

cut = ~np.isin(sources['idx'],[101,160,192,194,282])
sources = sources[cut]

# for idx in tqdm(sources['idx'],total=len(sources)):
#     j = np.argwhere(sources['idx'] == idx)[0][0]
#     age = sources['age'][j]
#     mh = sources['[Fe/H]'][j]
#     e_mh = sources['e_[Fe/H]'][j]
#     e_age = sources['e_age'][j]
    
#     age_vec = np.random.normal(age,e_age,int(1e3))
#     mh_vec = np.random.normal(mh,e_mh,int(1e3))
#     k_chain = []
#     for age,mh in zip(age_vec,mh_vec):
#         m_min = get_initial_mass(age,mh,phase='MSTO')
#         if np.isnan(m_min) or m_min > 8:
#             continue
#         m_max = 8
#         m_arr = sample_kroupa_uniform(m_min,m_max,n = 100)
#         k_chain.extend(m_arr)
#     np.save(f'../data/chains/kroupa_{idx}.npy',k_chain)

# uv = ~(np.isnan(sources['nuv_mag']) & np.isnan(sources['fuv_mag']))
# uv_excess = np.isin(sources['idx'],[53,174,175,180,236,249,281,283])
# no_excess = uv & ~uv_excess
# tbl = sources[uv]
# add_temperature_limits_to_table(sources)

# for idx in tqdm(sources['idx'][no_excess],total=len(sources[no_excess])):
#     j = np.argwhere(sources['idx'] == idx)[0][0]
#     age = sources['age'][j]
#     mh = sources['[Fe/H]'][j]
#     e_age = sources['e_age'][j]
#     e_mh = sources['e_[Fe/H]'][j]

#     age_vec = np.random.normal(age,e_age,int(1e3))
#     mh_vec = np.random.normal(mh,e_mh,int(1e3))
#     age_wd = get_cooling_age(sources['teff2_max'][j],sources['m2'][j])
#     t_chain = []
#     for age,mh in zip(age_vec,mh_vec):
#         age_ms = age - age_wd
#         if age_ms < 0:
#             continue
#         m_min = get_initial_mass(age_ms,mh,phase='AGB')
#         m_max = 8
#         if np.isnan(m_min) or m_min > 8:
#             continue
#         m_arr = sample_kroupa_uniform(m_min,m_max,n = 100)
#         t_chain.extend(m_arr)
#     np.save(f'../data/chains/trunc_kroupa_{idx}.npy',t_chain)


# uv_excess = [53,174,175,236,249,281,283]
# tbl = Table.read('../table_C.fits')

# for idx in tqdm(uv_excess,total = len(uv_excess)):
#     if path.exists(f'../data/chains/mchain_{idx}.csv'):
#         wmchain = weight_m_init_posterior(idx)
#         np.save(f'../data/chains/wmchain_{idx}.npy',wmchain['m_i'])
#         wmchain.write(f'../data/chains/wmchain_{idx}.csv',overwrite=True)
#     else:
#         print(f'No mchain for {idx}')

In [ ]:
## plot histograms
# sources = Table.read('../table_C.fits')
# for idx in sources['idx']:
#     if path.exists(f'../data/chains/wmchain_{idx}.npy'):
#         kchain = np.load(f'../data/chains/kroupa_{idx}.npy')
#         mchain = pd.read_csv(f'../data/chains/mchain_{idx}.csv')['m_i']
#         wmchain = np.load(f'../data/chains/wmchain_{idx}.npy')
#         mchain = mchain[mchain < 8]

#         fig,ax = plt.subplots(dpi = 300,layout='constrained')
#         if idx == 175 or idx == 249:
#             ax.hist(kchain,bins=30,histtype='step',color='RoyalBlue',label=f'Kroupa prior',density=True,range = (2,5),linestyle='dashed',lw=1.5)    
#         else:
#             ax.hist(kchain,bins=30,histtype='step',color='RoyalBlue',label=f'Kroupa prior',density=True,linestyle='dashed',lw=1.5)
        
#         ax.hist(mchain,bins=30,histtype='step',color='Crimson',label=f'Unweighted',density=True,linestyle='dashdot',lw=1.5)
#         ax.hist(wmchain,bins=30,histtype='step',color='k',label=f'Weighted,\ncandidate {idx}',density=True,zorder=5,linestyle='solid')
#         # ax.axvline(np.median(kchain),ls='--',color='b',label=f'50% = {np.median(kchain):.1f}M$_\odot$')

#         ax.set_xlabel(r'M$_{\text{initial}}\,($M$_{\odot}$)',fontsize=14,labelpad=0)
#         ax.set_ylabel('Normalized counts',fontsize=14)
#         ax.legend(fontsize=14)
#         fig.savefig(f'../img/mass_hist/mass_posterior_{idx}.png',bbox_inches='tight')
#         # fig.savefig(f'../img/kroupa_prior/kroupa_prior_{idx}.png',bbox_inches='tight')
#         plt.close()

# Plotting the IFMR

In [3]:
sources = Table.read('../table_C.fits')
cut = ~np.isin(sources['idx'],[101,160,192,194,282,114,118,121,48,174,282,283])
ifmr = create_ifmr_tbl(sources[cut])

100%|██████████| 26/26 [00:00<00:00, 103.65it/s]


In [ ]:
ifmr.show_in_notebook()

In [4]:
## No UV IFMR

import matplotlib.patches as mpatches

fig,ax = plt.subplots(dpi = 300,layout='constrained')

lower_error1 = ifmr['m_i_1'] - ifmr['m_i_min1']
upper_error1 = ifmr['m_i_max1'] - ifmr['m_i_1']
error1 = [lower_error1, upper_error1]
cut = np.isnan(ifmr['m_i_2']) & np.isnan(ifmr['m_i_3']) & (ifmr['idx'] != 121) & ~np.isin(ifmr['idx'],[101,160,180,192,194,282])
error1 = [lower_error1[cut], upper_error1[cut]]
xerror1 = ifmr[cut]['m2_err']


lower_error2 = ifmr['m_i_2'] - ifmr['m_i_min2']
upper_error2 = ifmr['m_i_max2'] - ifmr['m_i_2']
error2 = [lower_error2, upper_error2]
cut2 = ~np.isnan(ifmr['m_i_2']) & (ifmr['m_i_2'] > -1)
error2 = [lower_error2[cut2], upper_error2[cut2]]
xerror2 = ifmr[cut2]['m2_err']

lower_error3 = ifmr['m_i_3'] - ifmr['m_i_min3']
upper_error3 = ifmr['m_i_max3'] - ifmr['m_i_3']
error3 = [lower_error3, upper_error3]
cut3 = ~np.isnan(ifmr['m_i_3']) & (ifmr['m_i_3'] > -1) & (ifmr['idx'] != 174)
error3 = [lower_error3[cut3], upper_error3[cut3]]
xerror3 = ifmr[cut3]['m2_err']


kchains = []
for idx in ifmr[cut]['idx']:
    kchain = np.load(f'../data/chains/kroupa_{idx}.npy')
    kchains.append(kchain)

ebar = ax.errorbar(ifmr[cut]['m2'], ifmr[cut]['m_i_1'], xerr=xerror1,yerr=error1 ,fmt='s',label='No UV',
                   color='b',elinewidth=0.3,capsize=0,markersize=5,
            alpha=0.8,zorder=5)
ebar2 = ax.errorbar(ifmr[cut2]['m2'], ifmr[cut2]['m_i_2'], xerr=xerror2,yerr=error2 ,fmt='d',label='No Excess',
                   color='Crimson',elinewidth=0.3,capsize=0,markersize=5,
            alpha=0.8,zorder=5)
ebar3 = ax.errorbar(ifmr[cut3]['m2'], ifmr[cut3]['m_i_3'], xerr=xerror3,yerr=error3 ,fmt='o',label='UV Excess',
                     color='k',elinewidth=0.3,capsize=0,markersize=5,
                alpha=0.8,zorder=5)
# cummings = [inverse_cummings_ifmr(x) for x in xvec]
# cunningham = get_cunningham_ifmr_breakpoints()
# ax.plot(cunningham['M_final'],cunningham['M_initial'],color='Navy',label='Cunningham+2024',lw=1)
# ax.fill_betweenx(cunningham['M_initial'],cunningham['M_final_m1sig'],cunningham['M_final_p1sig'],color='Navy',alpha=0.3)
# ax.fill_betweenx(cunningham['M_initial'],cunningham['M_final_m2sig'],cunningham['M_final_p2sig'],color='Navy',alpha=0.15)

ax.set_xlabel(r'M$_{\text{final}}\,$(M$_{\odot}$)',fontsize=10,labelpad=0)
ax.set_ylabel(r'M$_{\text{initial}}\,$(M$_{\odot}$)',fontsize=10)
ax.set_ylim(top=8)
ax.minorticks_on()

ax.legend(loc='upper right')
# fig.savefig('../img/ifmr_no_uv.png')

In [ ]:
## No Excess IFMR

import matplotlib.patches as mpatches

ifmr.sort('m2')

lower_error1 = ifmr['m_i_1'] - ifmr['m_i_min1']
upper_error1 = ifmr['m_i_max1'] - ifmr['m_i_1']
error1 = [lower_error1, upper_error1]

lower_error2 = ifmr['m_i_2'] - ifmr['m_i_min2']
upper_error2 = ifmr['m_i_max2'] - ifmr['m_i_2']
error2 = [lower_error2, upper_error2]

cut = ~np.isnan(ifmr['m_i_2']) & (ifmr['m_i_2'] > -1)
error1 = [lower_error1[cut], upper_error1[cut]]
error2 = [lower_error2[cut], upper_error2[cut]]


kchains = []
tchains = []

for idx in ifmr[cut]['idx']:
    kchain = np.load(f'../data/chains/kroupa_{idx}.npy')
    tchain = np.load(f'../data/chains/trunc_kroupa_{idx}.npy')
    kchains.append(kchain)
    tchains.append(tchain)

fig, axs = plt.subplots(nrows=1, ncols=len(ifmr[cut]), dpi=300, layout='constrained', sharey=True)
axs[0].set_ylabel(r'M$_{\text{initial}}\,$(M$_{\odot}$)', fontsize=10)

for i in range(len(ifmr[cut])):
    idx = ifmr[cut]['idx'][i]
    ax = axs[i]

    # Blue violin (right side)
    kchain = kchains[i]
    parts = ax.violinplot(kchain, positions=[ifmr[cut]['m2'][i]], showmedians=False, showextrema=False, widths=0.03)
    for pc in parts['bodies']:
        path = pc.get_paths()[0]
        vertices = path.vertices
        violin_position = ifmr[cut]['m2'][i]
        vertices[:, 0] = np.maximum(vertices[:, 0], violin_position)
        pc.set_facecolor('RoyalBlue')
        pc.set_edgecolor('k')
        pc.set_alpha(0.5)

    # Red violin (left side)
    tchain = tchains[i]
    trunc_parts = ax.violinplot(tchain, positions=[ifmr[cut]['m2'][i]], showmedians=False, showextrema=False, widths=0.03)
    for pc in trunc_parts['bodies']:
        path = pc.get_paths()[0]
        vertices = path.vertices
        violin_position = ifmr[cut]['m2'][i]
        vertices[:, 0] = np.minimum(vertices[:, 0], violin_position)
        pc.set_facecolor('Crimson')
        pc.set_edgecolor('k')
        pc.set_alpha(0.5)
        pc.set_zorder(5)

    # Error bar
    ebar = ax.errorbar(
        ifmr[cut]['m2'][i],
        ifmr[cut]['m_i_2'][i],
        xerr=ifmr[cut]['m2_err'][i],
        fmt='d',
        color='Crimson',
        elinewidth=0.5,
        capsize=1,
        zorder=10,
        markersize=5,
        alpha=1,
    )

    # Add subplot-specific annotations
    j = np.argwhere(sources['idx'] == idx)[0][0]
    m1 = sources[j]['m1']
    period = sources[j]['period']
    eccentricity = sources[j]['eccentricity']
    cluster_age = sources[j]['age']
    ax.text(
        0.5,
        0.98,
        f'Candidate {idx}\n' +
        f'$M_1$={m1:.2f} M$_\odot$\nP={int(period)} days\ne={eccentricity:.2f}\n' +
        r'$\tau_\text{tot}$=' + f'{int(cluster_age)} Myr',
        transform=ax.transAxes,
        fontsize=8,
        verticalalignment='top',
        horizontalalignment='center',
        bbox=dict(facecolor='none', edgecolor='none', alpha=0.5)
    )

    # Adjust x-limits for each subplot
    # ax.set_xlim(ifmr[cut]['m2'][i] - 0.07, ifmr[cut]['m2'][i] + 0.07)
    ax.set_xlabel(r'M$_{\text{final}}\,$(M$_{\odot}$)', fontsize=10, labelpad=0)
    ax.set_xlim(ifmr[cut][i]['m2']-0.09,ifmr[cut][i]['m2']+0.09)
    ax.minorticks_on()
# Adjust y-axis limits for all subplots
axs[0].set_ylim(1.7, 10)

# fig.savefig('../img/ifmr_no_uv_excess.png')

In [7]:
## UV Excess IFMR

import matplotlib.patches as mpatches

ifmr.sort('m2')

lower_error3 = ifmr['m_i_3'] - ifmr['m_i_min3']
upper_error3 = ifmr['m_i_max3'] - ifmr['m_i_3']
error3 = [lower_error3, upper_error3]

cut = ~np.isnan(ifmr['m_i_3']) & (ifmr['m_i_3'] > -1) 
cut = cut & (ifmr['idx'] != 174)
error3 = [lower_error3[cut], upper_error3[cut]]

kchains = []
for idx in ifmr[cut]['idx']:
    kchain = np.load(f'../data/chains/kroupa_{idx}.npy')
    kchains.append(kchain)


fig,axs = plt.subplots(nrows=1,ncols=len(ifmr[cut]),dpi = 300,layout='constrained',sharey=True)
axs[0].set_ylabel(r'M$_{\text{initial}}\,$(M$_{\odot}$)',fontsize=10)
axs[0].set_ylim(1.7,10)
for i in range(len(ifmr[cut])):
    idx = ifmr[cut]['idx'][i]
    kchain = kchains[i]
    if len(kchain) > 0:
        ax = axs[i]
        parts = ax.violinplot(kchain,positions=[ifmr[cut][i]['m2']],showmedians=False,showextrema=False,widths=0.03)
        for pc in parts['bodies']:
            pc.set_facecolor('RoyalBlue')
            pc.set_edgecolor('k')
            pc.set_alpha(0.5)
        color = parts['bodies'][0].get_facecolor().flatten()
        violin = mpatches.Patch(color=color)
        ebar = ax.errorbar(ifmr[cut]['m2'][i], ifmr[cut]['m_i_3'][i], yerr=error3[0][i], xerr=ifmr[cut][i]['m2_err'],
                    fmt='.', color='k', elinewidth=0.5, capsize=1.5,
                    zorder=5, markersize=10,mec='None')
        ax.set_xlim(ifmr[cut]['m2'][i]-0.07,ifmr[cut]['m2'][i]+0.07)
        ax.set_xlabel(r'M$_{\text{final}}\,$(M$_{\odot}$)',fontsize=10,labelpad=0)
        ax.minorticks_on()
        j = np.argwhere(sources['idx'] == idx)[0][0]
        m1 = sources[j]['m1']
        period = sources[j]['period']
        eccentricity = sources[j]['eccentricity']
        cluster_age = sources[j]['age']
        ax.text(0.5, 0.98, f'Candidate {idx}\n'+f'$M_1$={m1:.2f} M$_\odot$\nP={int(period)} days\ne={eccentricity:.2f}\n'+r'$\tau_\text{tot}$='+f'{int(cluster_age)} Myr',
            transform=ax.transAxes, fontsize=8, verticalalignment='top', horizontalalignment='center', bbox=dict(facecolor='none',edgecolor='none', alpha=0.5))

fig.savefig('../img/ifmr_uv_excess.png')

In [5]:
## groups IFMR

ifmr['m_i_4'] = np.zeros(len(ifmr))
ifmr['m_i_min4'] = np.zeros(len(ifmr))
ifmr['m_i_max4'] = np.zeros(len(ifmr))

ifmr['m_i_4'] += np.array([ifmr[k]['m_i_1'] if c else 0 for k, c in enumerate(cut)])
ifmr['m_i_min4'] += np.array([ifmr[k]['m_i_min1'] if c else 0 for k, c in enumerate(cut)])
ifmr['m_i_max4'] += np.array([ifmr[k]['m_i_max1'] if c else 0 for k, c in enumerate(cut)])

ifmr['m_i_4'] += np.array([ifmr[k]['m_i_2'] if c else 0 for k, c in enumerate(cut2)])
ifmr['m_i_min4'] += np.array([ifmr[k]['m_i_min2'] if c else 0 for k, c in enumerate(cut2)])
ifmr['m_i_max4'] += np.array([ifmr[k]['m_i_max2'] if c else 0 for k, c in enumerate(cut2)])

ifmr['m_i_4'] += np.array([ifmr[k]['m_i_3'] if c else 0 for k, c in enumerate(cut3)])
ifmr['m_i_min4'] += np.array([ifmr[k]['m_i_min3'] if c else 0 for k, c in enumerate(cut3)])
ifmr['m_i_max4'] += np.array([ifmr[k]['m_i_max3'] if c else 0 for k, c in enumerate(cut3)])

ifmr['m_i_4'] = [m4 if m4>0 else np.nan for m4 in ifmr['m_i_4']]
ifmr['m_i_min4'] = [m4 if m4>0 else np.nan for m4 in ifmr['m_i_min4']]
ifmr['m_i_max4'] = [m4 if m4>0 else np.nan for m4 in ifmr['m_i_max4']]
error4_low = np.array(ifmr['m_i_4'] - ifmr['m_i_min4'])
error4_high = np.array(ifmr['m_i_max4'] - ifmr['m_i_4'])

ifmr['mass_loss'] = (ifmr['m_i_4'] - ifmr['m2'])/ifmr['m_i_4']
ifmr['mass_loss_err'] = ((ifmr['m2_err']/ifmr['m_i_4'])**2 + (error4_low/ifmr['m_i_4'])**2 * ifmr['mass_loss']**2)**0.5


group1 = (ifmr['m2'] < 0.5) & (cut | cut2 | cut3)
inverse_cummings = [inverse_cummings_ifmr(x) for x in ifmr['m2']]
group2 = (ifmr['m2'] >= 0.5) & (ifmr['m2'] < 0.8) & (ifmr['m_i_4'] > inverse_cummings) & ~np.isnan(inverse_cummings) & (cut | cut2 | cut3)
group3 = ~group1 & ~group2 & (cut | cut2 | cut3)

fig,ax = plt.subplots(dpi = 300,layout='constrained')
ebar = ax.errorbar(ifmr[group1]['m2'], ifmr[group1]['m_i_4'], xerr=ifmr[group1]['m2_err'], yerr=[error4_low[group1],error4_high[group1]],
                    fmt='s', color='b', elinewidth=0.4, capsize=0, markersize=5, label='Group 1')
ebar2 = ax.errorbar(ifmr[group2]['m2'], ifmr[group2]['m_i_4'], xerr=ifmr[group2]['m2_err'], yerr=[error4_low[group2],error4_high[group2]],
                     fmt='d', color='Crimson', elinewidth=0.4, capsize=0, markersize=5, label='Group 2')
ax.scatter(ifmr[group2]['m2'],ifmr[group2]['m_i_min_abs'],marker='_',color='Crimson',s=50)
ax.vlines(ifmr[group2]['m2'],ifmr[group2]['m_i_min_abs'],ifmr[group2]['m_i_min4'],color='Crimson',lw=0.2,linestyle='dashed',zorder=-1)
ebar3 = ax.errorbar(ifmr[group3]['m2'], ifmr[group3]['m_i_4'], xerr=ifmr[group3]['m2_err'], yerr=[error4_low[group3],error4_high[group3]],
                     fmt='o', color='k', elinewidth=0.4, capsize=0, markersize=5, label='Group 3')
cunningham = get_cunningham_ifmr_breakpoints()
ax.plot(cunningham['M_final'],cunningham['M_initial'],color='Navy',label='Cunningham+2024',lw=1)
ax.fill_betweenx(cunningham['M_initial'],cunningham['M_final_m1sig'],cunningham['M_final_p1sig'],color='Navy',alpha=0.2)
ax.fill_betweenx(cunningham['M_initial'],cunningham['M_final_m2sig'],cunningham['M_final_p2sig'],color='Navy',alpha=0.1)

ax.set_xlabel(r'M$_{\text{final}}\,$(M$_{\odot}$)',fontsize=10,labelpad=0)
ax.set_ylabel(r'M$_{\text{initial}}\,$(M$_{\odot}$)',fontsize=10)
ax.set_ylim(top=9.6)
ax.minorticks_on()

ax.legend(loc='upper right')
# fig.savefig('../img/ifmr_groups.png')


In [47]:
## group 3 IFMR as MS+WD+WD systems

mswdwd = np.isin(ifmr['idx'],[110,154,183,260,262])
group3_a = group3 & ~mswdwd
cunningham = get_cunningham_ifmr_breakpoints()
wd2 = np.interp(ifmr[mswdwd]['m_i_4'],cunningham['M_initial'],cunningham['M_final'])
wd1 = ifmr[mswdwd]['m2'] - wd2


fig,ax = plt.subplots(dpi = 300,layout='constrained')
ax.plot(cunningham['M_final'],cunningham['M_initial'],color='Navy',label='Cunningham+2024',lw=1)
ax.fill_betweenx(cunningham['M_initial'],cunningham['M_final_m1sig'],cunningham['M_final_p1sig'],color='Navy',alpha=0.2)
ax.fill_betweenx(cunningham['M_initial'],cunningham['M_final_m2sig'],cunningham['M_final_p2sig'],color='Navy',alpha=0.1)


ax.hlines(ifmr[mswdwd]['m_i_4'],wd2,ifmr[mswdwd]['m2'],color='k',lw=0.3,ls='dotted')
ax.scatter(ifmr[mswdwd]['m2'],ifmr[mswdwd]['m_i_4'],label='WD$_1$+WD$_2$',color='k',s=20)
# ax.errorbar(ifmr[group3_a]['m2'], ifmr[group3_a]['m_i_4'], xerr=ifmr[group3_a]['m2_err'], yerr=[error4_low[group3_a],error4_high[group3_a]],
#                         fmt='o', color='k', elinewidth=0.4, capsize=0, markersize=5)
# ax.scatter(ifmr[group3_a]['m2'],ifmr[group3_a]['m_i_4'],label='Group 3',color='k',s=20)
ax.hlines(ifmr[mswdwd]['m_i_4'],wd1,wd2,color='k',lw=0.5,ls='dashed')
ax.scatter(wd1,ifmr[mswdwd]['m_i_4'],marker='*',color='k',s=50,label='WD$_1$',zorder=5)
ax.scatter(wd2,ifmr[mswdwd]['m_i_4'],marker='x',color='k',s=50,label='WD$_2$',zorder=5)

for idx in ifmr[mswdwd]['idx']:
    j = np.argwhere(ifmr['idx'] == idx)[0][0]
    k = np.argwhere(ifmr[mswdwd]['idx'] == idx)[0][0]
    m2 = ifmr[j]['m2']
    yshift = -0.3 if idx==110 else -0.15
    ax.text(m2 + 0.09, ifmr[j]['m_i_4']+yshift, f'{idx}', fontsize=10, verticalalignment='bottom', horizontalalignment='right')
    

ax.set_xlabel(r'M$_{\text{final}}\,$(M$_{\odot}$)',fontsize=14,labelpad=0)
ax.set_ylabel(r'M$_{\text{initial}}\,$(M$_{\odot}$)',fontsize=14)
ax.tick_params(axis='both', which='major', labelsize=14)
ax.minorticks_on()
ax.legend(loc='upper left',fontsize=12)
fig.savefig('../img/ifmr_triples.png')

# Correlations and more fun

In [41]:
## correlations between mass loss and system properties

# fig,ax = plt.subplots(dpi = 300,layout='constrained')

# ax.errorbar(ifmr[group1]['m2'], ifmr[group1]['eccentricity'], yerr=ifmr[group1]['eccentricity_error'], xerr = ifmr[group1]['m2_err'],
#              fmt='s', color='b', elinewidth=0.5, capsize=0, markersize=3, label='Group 1')
# ax.errorbar(ifmr[group2]['m2'], ifmr[group2]['eccentricity'], yerr=ifmr[group2]['eccentricity_error'], xerr = ifmr[group2]['m2_err'],
#              fmt='d', color='Crimson', elinewidth=0.5, capsize=0, markersize=3, label='Group 2')
# ax.errorbar(ifmr[group3]['m2'], ifmr[group3]['eccentricity'], yerr=ifmr[group3]['eccentricity_error'], xerr = ifmr[group3]['m2_err'],
#              fmt='o', color='k', elinewidth=0.5, capsize=0, markersize=3, label='Group 3')

# ax.minorticks_on()
# ax.set_xlabel(r'M$_{\text{final}}\,$(M$_{\odot}$)',fontsize=10,labelpad=0)
# ax.set_ylabel(r'Eccentricity',fontsize=10)
# ax.legend()
# fig.savefig('../img/eccentricity_groups.png')

# ax.errorbar(ifmr[group1]['[Fe/H]'], ifmr[group1]['mass_loss'], yerr=ifmr[group1]['mass_loss_err'], xerr=ifmr[group1]['e_[Fe/H]'],
#              fmt='s', color='b', elinewidth=0.5, capsize=0, markersize=3, label='Group 1')
# ax.errorbar(ifmr[group2]['[Fe/H]'], ifmr[group2]['mass_loss'], yerr=ifmr[group2]['mass_loss_err'], xerr=ifmr[group2]['e_[Fe/H]'],
#              fmt='d', color='Crimson', elinewidth=0.5, capsize=0, markersize=3, label='Group 2')
# ax.errorbar(ifmr[group3]['[Fe/H]'], ifmr[group3]['mass_loss'], yerr=ifmr[group3]['mass_loss_err'], xerr=ifmr[group3]['e_[Fe/H]'],
#              fmt='o', color='k', elinewidth=0.5, capsize=0, markersize=3, label='Group 3')

# ax.minorticks_on()
# ax.set_xlabel(r'[Fe/H]', fontsize=10, labelpad=0)
# ax.set_ylabel(r'Fractional Mass Loss', fontsize=10)
# ax.legend()
# fig.savefig('../img/mass_loss_vs_metallicity.png')


In [ ]:
## more correlations

# from scipy.optimize import curve_fit
# from scipy.stats import chi2

# cut = np.isnan(ifmr['m_i_2']) & np.isnan(ifmr['m_i_3']) & (ifmr['idx'] != 121) & ~np.isin(ifmr['idx'],[101,160,180,192,194,282])
# cut2 = ~np.isnan(ifmr['m_i_2']) & (ifmr['m_i_2'] > -1)
# cut3 = ~np.isnan(ifmr['m_i_3']) & (ifmr['m_i_3'] > -1) & (ifmr['idx'] != 174)


# mh1 = [sources[sources['idx'] == j]['[Fe/H]'][0] for j in ifmr[cut]['idx']]
# e_mh1 = [sources[sources['idx'] == j]['e_[Fe/H]'][0] for j in ifmr[cut]['idx']]
# p1 = [sources[sources['idx'] == j]['period'][0] for j in ifmr[cut]['idx']]
# e1 = [sources[sources['idx'] == j]['eccentricity'][0] for j in ifmr[cut]['idx']]
# m_i1 = ifmr[cut]['m_i_1']
# m_fin1 = ifmr[cut]['m2']
# m_prim1 = ifmr[cut]['m1']
# # e_m_i1 = (ifmr[cut]['m_i_max1'] - ifmr[cut]['m_i_min1'])/2
# e_m_i1 = ifmr[cut]['m_i_1'] - ifmr[cut]['m_i_min1'] # The error is asymmetric. This is the smaller error.
# f1 = (ifmr[cut]['m_i_1'] - ifmr[cut]['m2'])/ifmr[cut]['m_i_1']
# e_f1 = np.sqrt(f1**2*(e_m_i1/m_i1)**2 + (ifmr[cut]['m2_err']/m_i1)**2)

# mh2 = [sources[sources['idx'] == j]['[Fe/H]'][0] for j in ifmr[cut2]['idx']]
# e_mh2 = [sources[sources['idx'] == j]['e_[Fe/H]'][0] for j in ifmr[cut2]['idx']]
# p2 = [sources[sources['idx'] == j]['period'][0] for j in ifmr[cut2]['idx']]
# e2 = [sources[sources['idx'] == j]['eccentricity'][0] for j in ifmr[cut2]['idx']]
# m_i2 = ifmr[cut2]['m_i_2']
# m_fin2 = ifmr[cut2]['m2']
# m_prim2 = ifmr[cut2]['m1']
# # e_m_i2 = (ifmr[cut2]['m_i_max2'] - ifmr[cut2]['m_i_min2'])/2
# e_m_i2 = ifmr[cut2]['m_i_2'] - ifmr[cut2]['m_i_min2']
# f2 = (ifmr[cut2]['m_i_2'] - ifmr[cut2]['m2'])/ifmr[cut2]['m_i_2']
# e_f2 = np.sqrt(f2**2*(e_m_i2/m_i2)**2 + (ifmr[cut2]['m2_err']/m_i2)**2)

# mh3 = [sources[sources['idx'] == j]['[Fe/H]'][0] for j in ifmr[cut3]['idx']]
# e_mh3 = [sources[sources['idx'] == j]['e_[Fe/H]'][0] for j in ifmr[cut3]['idx']]
# p3 = [sources[sources['idx'] == j]['period'][0] for j in ifmr[cut3]['idx']]
# e3 = [sources[sources['idx'] == j]['eccentricity'][0] for j in ifmr[cut3]['idx']]
# m_i3 = ifmr[cut3]['m_i_3']
# m_fin3 = ifmr[cut3]['m2']
# m_prim3 = ifmr[cut3]['m1']
# # e_m_i3 = (ifmr[cut3]['m_i_max3'] - ifmr[cut3]['m_i_min3'])/2
# e_m_i3 = ifmr[cut3]['m_i_3'] - ifmr[cut3]['m_i_min3']
# f3 = (ifmr[cut3]['m_i_3'] - ifmr[cut3]['m2'])/ifmr[cut3]['m_i_3']
# e_f3 = np.sqrt(f3**2*(e_m_i3/m_i3)**2 + (ifmr[cut3]['m2_err']/m_i3)**2)


# f = np.concatenate([f1,f2,f3]) ## fractional mass loss
# e_f = np.concatenate([e_f1,e_f2,e_f3]) ## fractional mass loss error- ideal case
# mh = np.concatenate([mh1,mh2,mh3]) ## metallicity
# e_mh = np.concatenate([e_mh1,e_mh2,e_mh3]) ## metallicity error
# m_fin = np.concatenate([m_fin1,m_fin2,m_fin3]) ## final mass
# m_i = np.concatenate([m_i1,m_i2,m_i3]) ## initial mass
# m_prim = np.concatenate([m_prim1,m_prim2,m_prim3]) ## primary mass
# p = np.concatenate([p1,p2,p3]) ## period
# e = np.concatenate([e1,e2,e3]) ## eccentricity

# q_today = m_prim/m_fin ## mass ratio today, M1/M2, we flip the previous notation of M2/M1 to calculate the relevant roche lobe radius
# sep_today = (p/365)**(2/3) * (m_prim + m_fin)**(1/3) ## separation today in AU
# q_init_wind = m_prim/m_i ## mass ratio at the start of mass transfer, assuming accretor mass unchanged: M_1/M_init
# sep_wind = (m_prim+m_fin)/(m_prim+m_i) * sep_today ## separation at the start of mass transfer in AU assuming no accretion

# fig,ax = plt.subplots(dpi = 300,layout='constrained')
# ax.scatter(sep_today,f,marker='o',label='Today')
# ax.scatter(sep_wind,f,marker='d',label='Start of mass transfer')
# ax.set_xlabel('Separation (AU)')
# ax.set_ylabel(r'$\frac{M_i - M_f}{M_i}$')
# ax.legend()

# constant = np.average(f,weights=1/e_f**2)
# constant_err = np.sqrt(1/np.sum(1/e_f**2))

# chisquared = np.sum((f - constant)**2/e_f**2)
# pval = chi2.sf(chisquared,len(f)-1)


# def linear(x,m,c):
#     return m*x + c

# popt,pcov = curve_fit(linear,mh,f,sigma=e_f,absolute_sigma=True)
# m,c = popt
# m_err,c_err = np.sqrt(np.diag(pcov))

# chisquared_lin = np.sum((f - linear(mh,m,c))**2/e_f**2)
# pval_lin = chi2.sf(chisquared_lin,len(f)-2)

# print(chisquared , len(f) - 1)
# print(chisquared_lin, len(f) - 2)
# # Calculate Akaike Information Criterion (AIC)
# aic_constant = 2 * 1 + chisquared + (2 * 1 * (1 + 1)) / (len(f) - 1 - 1)
# aic_linear = 2 * 2 + chisquared_lin + (2 * 2 * (2 + 1)) / (len(f) - 2 - 1)

# print(f"AIC (constant model): {aic_constant}")
# print(f"AIC (linear model): {aic_linear}")